In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
# import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
basics_cols = [
    'item_price_level', 'item_sales_level', 'item_collected_level',
    'item_pv_level', 'user_gender_id', 'user_age_level', 'user_occupation_id',
    'user_star_level', 'context_page_id', 'shop_review_num_level',
    'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
    'shop_score_delivery', 'shop_score_description'
]

zero_feas = [
    # 基础特征
    basics_cols,
    #id转vector
    [
        'item_id_idx', 'user_id_idx', 'shop_id_idx', 'item_brand_id_idx',
        'item_city_id_idx'
    ],
    # 时间特征 !!
    [
        'day', 'hour', 'minute', 'yesterday', 'yesterhour', 'next_time_sub',
        'last_time_sub', 'user_item_next_time_sub', 'user_item_last_time_sub',
        'user_shop_next_time_sub', 'user_shop_last_time_sub',
        'user_count_first_time_sub', 'user_item_count_first_time_sub',
        'user_shop_count_first_time_sub', 'is_last_click',
        'is_last_user_item_click', 'max_click_time_sub',
        'max_user_item_click_time_sub'
    ],
    #shop、item得分 !!
    ['item_score', 'item_score2', 'item_score3', 'shop_score'],
    # 查询次数 !
    [
        'user_day_query', 'user_yesterday_query', 'user_hour_query',
        'user_yesterhour_query', 'user_minute_query', 'item_day_query',
        'item_hour_query', 'user_item_day_query', 'user_item_hour_query',
        'item_minute_query', 'user_item_minute_query'
    ],
    # user在一个时间段出现的次数序号 !
    ['user_count', 'user_day_count', 'user_hour_count', 'user_minute_count'],
    # item与pred类型特征
    [
        'item_category_split_count', 'item_property_split_count',
        'pred_category_split_count', 'pred_property_split_count',
        'item_pred_category_score', 'item_pred_property_score',
        'item_pred_category_score_item%', 'item_pred_property_score_item%',
        'item_pred_category_score_pred%', 'item_pred_property_score_pred%'
    ],
    # 统计特征
    [
        'user_shop_count', 'user_item_count', 'user_context_count',
        'user_shop_trade_count', 'user_item_trade_count',
        'user_context_trade_count', 'user_brand_count', 'user_city_count',
        'user_brand_trade_count', 'user_city_trade_count', 'shop_item_count',
        'item_occupation_count', 'item_age_count', 'item_gender_count',
        'item_occupation_trade_count', 'item_age_trade_count',
        'item_gender_trade_count', 'brand_item_count', 'city_brand_count',
        'city_item_count', 'user_item_num', 'user_shop_num',
        'user_item_day_num', 'user_shop_day_num', 'min_time'
    ],
    # 火热程度（搜索次数） --
    [
        'item_hot', 'user_hot', 'shop_hot', 'brand_hot', 'occupation_hot',
        'city_hot', 'item_trade_hot', 'user_trade_hot', 'shop_trade_hot',
        'brand_trade_hot', 'occupation_trade_hot', 'city_trade_hot'
    ],
    # user购买力、成交率 --
    [
        'user_sell_power', 'user_sell_trade_power', 'user_sell_power_mean',
        'user_sell_trade_power_mean', 'user_sell_power_mean_item_minus',
        'user_sell_trade_power_mean_item_minus', 'is_home'
    ],
    # 有没有在历史中出现过 --
    [
        'user_is_his', 'brand_is_his', 'user_is_trade_his',
        'item_is_trade_his', 'brand_is_trade_his', 'item_is_his'
    ],
    # 成交率 --
    [
        'user_trade_percent',
        'occupation_trade_percent',
        'brand_trade_percent',
        'item_trade_percent',
    ],
    # 历史数据中出现次数 --
    [
        'city_brand_count_his', 'city_item_count_his', 'shop_item_count_his',
        'brand_item_count_his'
    ],
    # star用户喜爱程度
    [],
    # 平均值编码
    [],
    # 贝叶斯平滑
    [],
    # 分级特征
    [],
    # 最大最小特征/局部最优特征
    []
]

cjf_feas = [
    # 基础特征
    [],
    # 时间特征
    [],
    #shop、item得分
    [],
    # 查询次数
    [],
    # user在一个时间段出现的次数序号
    [],
    # item与pred类型特征
    [
        'category_1',
        'category_2',
        'property_0',
        'property_1',
        'property_2',
        'property_3',
        'property_4',
        'property_5',
        'property_6',
        'property_7',
        'property_8',
        'query_col_0',
        'query_col_1',
        'query_col_2',
        'query_col_3',
        'query_col_4',
        'query_col_5',
        'query_col_6',
    ],
    # 统计特征
    [],
    # 火热程度（搜索次数）
    [],
    # user购买力、成交率
    [],
    # 有没有在历史中出现过
    [],
    # 成交率
    [],
    # 历史数据中出现次数
    [],
    # star用户喜爱程度
    [],
    # 平均值编码
    [],
    # 贝叶斯平滑
    [
        'item_pv_stair_shop_id_bayes_rate_2',
        'item_city_id_shop_id_bayes_rate_2',
        'item_brand_id_shop_review_num_level_bayes_rate_1',
        'item_sales_stair_user_age_stair_bayes_rate_1',
        'time_slice_shop_review_num_level_bayes_rate_1',
        'item_collected_level_bayes_rate_1'
    ],
    # 分级特征
    [
        'gender_filled',
        'user_age_stair',
        'user_occupation_stair',
        'user_star_stair',
        'context_page_stair',
        'hour_stair',
        'item_price_stair',
        'item_sales_stair',
    ],
    # 最大最小特征/局部最优特征
    [
        'user_query_cheapest',
        'user_query_maxsell',
        'user_query_maxcollect',
        'user_query_maxpv',
        'user_query_best_service',
        'user_query_best_delivery',
        'user_query_best_description',
        'user_query_maxshopstar',
        'user_query_maxreview',
        'user_query_maxqueryitem_c_similarity',
        'user_query_maxqueryitem_p_similarity',
    ]
]

cjy_feas = [
    # 基础特征
    [],
    # 时间特征
    [],
    #shop、item得分
    [],
    # 查询次数
    [],
    # user在一个时间段出现的次数序号
    [],
    # item与pred类型特征
    [
        'item_cat_vec_0', 'item_cat_vec_1', 'item_cat_vec_2', 'item_cat_vec_3',
        'item_cat_vec_4', 'item_cat_vec_5', 'item_cat_vec_6', 'item_cat_vec_7',
        'item_cat_vec_8', 'item_cat_vec_9', 'item_cat_vec_10',
        'item_cat_vec_11', 'item_cat_vec_12', 'item_cat_vec_13',
        'item_cat_vec_14', 'item_cat_vec_15', 'item_cat_vec_16',
        'item_cat_vec_17', 'item_cat_vec_18', 'item_cat_vec_19',
        'item_cat_vec_20', 'item_cat_vec_21', 'item_cat_vec_22',
        'item_cat_vec_23', 'pred_cat_mean_0', 'pred_cat_mean_1',
        'pred_cat_mean_2', 'pred_cat_mean_3', 'pred_cat_mean_4',
        'pred_cat_mean_5', 'pred_cat_mean_6', 'pred_cat_mean_7',
        'pred_cat_mean_8', 'pred_cat_mean_9', 'pred_cat_mean_10',
        'pred_cat_mean_11', 'pred_cat_mean_12', 'pred_cat_mean_13',
        'pred_cat_mean_14', 'pred_cat_mean_15', 'pred_cat_mean_16',
        'pred_cat_mean_17', 'pred_cat_mean_18', 'pred_cat_mean_19',
        'pred_cat_mean_20', 'pred_cat_mean_21', 'pred_cat_mean_22',
        'pred_cat_mean_23', 'item_pred_cat_cos', 'item_pred_ppt_cos'
    ],
    # 统计特征
    ['shop_item_cnt', 'shop_brand_cnt'],
    # 火热程度（搜索次数）
    ['24h_cat_hot', '24h_ppt_hot', '24h_user_seem_times'],
    # user购买力、成交率
    [],
    # 有没有在历史中出现过
    [],
    # 成交率
    [],
    # 历史数据中出现次数
    [],
    # star用户喜爱程度
    [
        'star_category_max', 'star_category_min', 'star_category_mean',
        'star_category_var', 'star_property_max', 'star_property_min',
        'star_property_mean', 'star_property_var'
    ],
    # 平均值编码
    [
        'item_brand_id+item_price_level_pred_1', 'shop_id_pred_1',
        'item_id_pred_1', 'item_sales_level+item_collected_level_pred_1',
        'shop_star_level+shop_item_cnt_pred_1',
        'shop_review_num_level+shop_item_cnt_pred_1',
        'item_price_level+item_pv_level_pred_1', 'item_price_level_pred_1',
        'item_sales_level+item_pv_level_pred_1',
        'shop_item_cnt+shop_brand_cnt_pred_1',
        'user_age_level+user_star_level_pred_1', '24h_user_seem_times_pred_1',
        'item_brand_id+shop_brand_cnt_pred_1',
        'item_collected_level+item_id_pred_1',
        'item_price_level+item_pred_cat_cos_pred_1',
        'item_brand_id+hist_hour_ctr_pred_1',
        'item_brand_id+shop_score_description_pred_1',
        'item_city_id+context_page_id_pred_1',
        'item_pv_level+shop_review_positive_rate_pred_1',
        'item_id+hist_hour_ctr_pred_1',
        'item_price_level+star_property_min_pred_1',
        'user_star_level+shop_id_pred_1', 'shop_item_cnt+24h_ppt_hot_pred_1',
        'item_brand_id+star_property_max_pred_1',
        'user_gender_id+star_category_min_pred_1',
        'shop_item_cnt+star_property_max_pred_1',
        'item_sales_level+context_page_id_pred_1',
        'item_collected_level+shop_score_description_pred_1',
        'shop_review_num_level+hist_hour_ctr_pred_1',
        'item_collected_level+user_gender_id_pred_1',
        'shop_item_cnt+star_property_var_pred_1',
        '24h_ppt_hot+star_property_max_pred_1',
        'user_star_level+star_property_max_pred_1',
        'user_age_level+user_id_pred_1'
    ],
    # 贝叶斯平滑
    [],
    # 分级特征
    [],
    # 最大最小特征/局部最优特征
    []
]

target = ['is_trade']

In [3]:
zero_fea_list = []
for crt_list in zero_feas:
    for i in crt_list:
        zero_fea_list.append(i)
zero_fea_list = list(set(zero_fea_list))
print('zero_feas len =', len(zero_fea_list))

cjf_fea_list = []
for crt_list in cjf_feas:
    for i in crt_list:
        cjf_fea_list.append(i)
cjf_fea_list = list(set(cjf_fea_list))
print('cjf_feas len =', len(cjf_fea_list))

cjy_fea_list = []
for crt_list in cjy_feas:
    for i in crt_list:
        cjy_fea_list.append(i)
cjy_fea_list = list(set(cjy_fea_list))
print('cjy_feas len =', len(cjy_fea_list))

zero_feas len = 125
cjf_feas len = 43
cjy_feas len = 97


In [4]:
train = pd.read_csv('train_preprocessing34_full.csv')

train.shape
train.head()

(278894, 266)

,user_context_trade_count,user_city_trade_count,user_day_count,user_shop_count,city_item_count_his,shop_item_count_his,user_shop_next_time_sub,user_age_level,city_item_count,shop_star_level,user_trade_hot,user_item_num,shop_trade_hot,user_count,shop_score_delivery,user_item_last_time_sub,city_hot,user_sell_trade_power_mean,is_last_click,user_is_his,is_last_user_item_click,item_hot,user_item_count_first_time_sub,user_sell_power,user_shop_day_num,item_trade_hot,item_sales_level,item_occupation_trade_count,item_gender_count,minute,hour,item_score2,user_minute_query,user_id_idx,item_pred_property_score,brand_item_count_his,user_minute_count,pred_property_split_count,item_occupation_count,user_shop_trade_count,item_brand_id_idx,next_time_sub,user_day_query,user_sell_power_mean_item_minus,yesterday,user_item_trade_count,user_yesterhour_query,brand_is_his,item_pred_category_score,user_brand_count,user_item_count,city_brand_count_his,item_gender_trade_count,user_shop_count_first_time_sub,brand_hot,item_trade_percent,user_is_trade_his,item_collected_level,item_id_idx,user_sell_power_mean,brand_is_trade_his,city_trade_hot,user_gender_id,city_brand_count,item_minute_query,item_is_his,item_pred_property_score_item%,occupation_hot,context_page_id,shop_id_idx,user_context_count,last_time_sub,item_property_split_count,item_hour_query,user_shop_last_time_sub,user_trade_percent,min_time,shop_item_count,user_count_first_time_sub,user_item_day_query,user_sell_trade_power,shop_hot,user_occupation_id,user_brand_trade_count,brand_trade_hot,user_shop_num,user_item_minute_query,item_city_id_idx,day,item_pv_level,user_hot,user_item_next_time_sub,item_category_split_count,shop_score,brand_item_count,user_hour_query,pred_category_split_count,item_score,user_hour_count,brand_trade_percent,item_price_level,item_age_count,user_yesterday_query,shop_score_service,occupation_trade_hot,user_city_count,item_score3,user_item_day_num,item_pred_property_score_pred%,item_is_trade_his,item_pred_category_score_item%,occupation_trade_percent,shop_review_num_level,shop_review_positive_rate,user_sell_trade_power_mean_item_minus,user_item_hour_query,shop_score_description,yesterhour,max_click_time_sub,is_home,item_pred_category_score_pred%,item_age_trade_count,user_star_level,item_day_query,max_user_item_click_time_sub,user_query_maxsell,category_2,property_7,property_1,query_col_6,item_collected_level_bayes_rate_1,item_price_stair,user_query_maxpv,user_query_cheapest,user_occupation_stair,property_5,user_query_best_service,user_age_stair,query_col_5,query_col_1,query_col_0,user_query_maxqueryitem_c_similarity,user_star_stair,item_pv_stair_shop_id_bayes_rate_2,item_sales_stair,property_8,user_query_maxcollect,property_2,user_query_best_description,user_query_maxreview,property_0,property_3,query_col_2,category_1,hour_stair,user_query_best_delivery,query_col_3,user_query_maxshopstar,time_slice_shop_review_num_level_bayes_rate_1,property_6,item_sales_stair_user_age_stair_bayes_rate_1,item_city_id_shop_id_bayes_rate_2,context_page_stair,user_query_maxqueryitem_p_similarity,item_brand_id_shop_review_num_level_bayes_rate_1,gender_filled,query_col_4,property_4,pred_cat_mean_5,pred_cat_mean_14,pred_cat_mean_17,pred_cat_mean_22,shop_item_cnt,item_cat_vec_22,item_cat_vec_2,item_brand_id+hist_hour_ctr_pred_1,item_brand_id+shop_score_description_pred_1,user_gender_id+star_category_min_pred_1,pred_cat_mean_15,item_id_pred_1,item_cat_vec_21,24h_ppt_hot+star_property_max_pred_1,star_category_mean,item_cat_vec_19,item_price_level+item_pred_cat_cos_pred_1,star_property_min,24h_cat_hot,item_cat_vec_5,item_cat_vec_3,item_cat_vec_16,pred_cat_mean_0,star_property_var,shop_review_num_level+shop_item_cnt_pred_1,pred_cat_mean_6,item_pv_level+shop_review_positive_rate_pred_1,item_cat_vec_9,item_sales_level+item_pv_level_pred_1,pred_cat_mean_2,pred_cat_mean_8,item_pred_cat_cos,item_price_level+star_property_min_pred_1,item_pred_ppt_cos,item_cat_vec_10,pred_cat_mean_19,item_cat_vec_13,shop_brand_

In [6]:
test = train[train.day == 25]
val = train[train.day == 24]
train = train[(train.day != 24) & (train.day != 25)]

train.shape
val.shape
test.shape

(203112, 266)

(57411, 266)

(18371, 266)

In [7]:
import sys
sys.path.append('/Users/zero/xgboost/python-package')
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
rubbish_features_zero = [
    'user_city_trade_count', 'user_id_idx', 'brand_item_count',
    'brand_trade_percent', 'item_score', 'shop_item_count_his',
    'item_occupation_count', 'item_age_count', 'shop_id_idx',
    'item_brand_id_idx', 'item_hour_query', 'user_trade_percent',
    'user_city_count', 'min_time', 'user_sell_trade_power_mean', 'item_score3',
    'user_item_day_query', 'user_yesterhour_query', 'brand_trade_hot',
    'item_trade_hot', 'item_gender_trade_count',
    'item_pred_category_score_pred%', 'item_age_trade_count',
    'item_gender_count', 'item_day_query', 'item_id_idx'
]
rubbish_features_cjf = [
    'category_2', 'item_sales_stair_user_age_stair_bayes_rate_1', 'property_5',
    'property_4', 'property_0', 'category_1',
    'item_collected_level_bayes_rate_1', 'query_col_5', 'property_2',
    'item_pv_stair_shop_id_bayes_rate_2',
    'time_slice_shop_review_num_level_bayes_rate_1', 'property_3',
    'query_col_2', 'property_8', 'query_col_0', 'query_col_1', 'query_col_3',
    'property_6', 'item_brand_id_shop_review_num_level_bayes_rate_1',
    'query_col_6', 'query_col_4', 'property_1', 'property_7',
    'item_city_id_shop_id_bayes_rate_2'
]

rubbish_features_cv = [
    'item_pred_cat_cos', 'item_occupation_count', 'property_1',
    'item_pred_ppt_cos', 'property_7',
    'item_price_level+star_property_min_pred_1',
    'user_star_level+star_property_max_pred_1',
    'item_pv_stair_shop_id_bayes_rate_2', 'item_brand_id_idx',
    'star_category_mean', '24h_ppt_hot+star_property_max_pred_1',
    'item_sales_stair_user_age_stair_bayes_rate_1',
    'item_price_level+item_pv_level_pred_1', 'property_5', 'category_1',
    'shop_item_cnt+star_property_max_pred_1',
    'time_slice_shop_review_num_level_bayes_rate_1', 'user_id_idx',
    'shop_item_cnt+24h_ppt_hot_pred_1',
    'item_sales_level+context_page_id_pred_1',
    'item_city_id_shop_id_bayes_rate_2', 'property_4', 'property_8',
    'item_brand_id_shop_review_num_level_bayes_rate_1', 'item_age_count',
    'item_collected_level_bayes_rate_1'
]

rubbish_features = [
    'user_id_idx', 'item_id_idx', 'item_hour_query',
    'item_pred_category_score_pred%', 'item_brand_id_idx', 'brand_trade_hot',
    'brand_trade_hot', 'shop_id_idx', 'min_time', 'item_age_count',
    'brand_trade_percent'
]

In [9]:
predictors = list(
    set(zero_fea_list + cjy_fea_list + cjf_fea_list) - set(basics_cols))
predictors = list(set(basics_cols + predictors))
predictors = list(
    set(predictors) -
    set(rubbish_features_zero + rubbish_features_cjf + rubbish_features_cv))
len(predictors)

205

In [13]:
#train/val
params = {
    'objective': 'binary:logistic',
    'eta': 0.06,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': -1
}

# feature_performance = {}

for i in tqdm(predictors):
    temp_features = predictors.copy()
#     temp_features.remove(i)
#     len(temp_features)
    xgbtrain = xgb.DMatrix(train[temp_features], train[target])
    xgbval = xgb.DMatrix(val[temp_features])
    model = xgb.train(params, xgbtrain, num_boost_round=450)
    y_pred = model.predict(xgbval)
    y_pred = pd.Series(y_pred, name='val')

    print(str(i) + ':' + str(log_loss(val[target], y_pred)))
#     feature_performance[i] = log_loss(val[target], y_pred)

  0%|          | 1/205 [02:51<9:43:27, 171.60s/it]

item_pred_category_score:0.0792309261243


  1%|          | 2/205 [05:41<9:38:20, 170.94s/it]

shop_trade_hot:0.0792309261243


  1%|▏         | 3/205 [08:31<9:34:32, 170.66s/it]

...

  3%|▎         | 7/205 [17:46<8:22:56, 152.41s/it]

item_cat_vec_22:0.0792309261243


  4%|▍         | 8/205 [19:19<7:56:04, 145.00s/it]

pred_cat_mean_6:0.0792309261243


KeyboardInterrupt: 

In [16]:
baseline = 0.0792309261243
rubbish_features = []

for i in enumerate(feature_performance):
    if (feature_performance[i[1]] > baseline):
        rubbish_features.append(i[1])
        
len(rubbish_features)

162

In [137]:
#cv
params = {
    'objective': 'binary:logistic',
    'eta': 0.06,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}
# temp_predictors = list(
#     set(zero_fea_list) - set(basics_cols))
# temp_predictors = list(set(basics_cols + temp_predictors))
# temp_predictors = list(
#     set(temp_predictors) -
#     set(rubbish_features_zero + rubbish_features_cv))
# len(temp_predictors)

cv_train = pd.concat([train, val])
xgbtrain = xgb.DMatrix(cv_train[good_features + basics_cols], cv_train[target])
len(good_features)

cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=500,
    nfold=5,
    early_stopping_rounds=50,
    seed=201803)
print(str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))

# 0.0836343333333(zero+cjf+cjy)
# 0.083560(zero+cjf+cjy - rubbish)
# 0.083543(zero - rubbish_zero)
# 0.083598(zero - rubbish_zero - rubbish_cv)
# 0.0835596666667(zero+cjy - rubbish_zero)
# 0.0838413333333(zero+cjf - rubbish_zero)
# 0.0836076666667(zero+cjf+cjy - rubbish_zero)
# 0.0836136666667(zero+cjf+cjy - rubbish_cv)
# 0.083562(zero+cjf+cjy - rubbish_zero - rubbish_cjf)
# 0.083492(zero+cjf+cjy - rubbish_zero - rubbish_cjf - rubbish_cv)

# 0.0836006666667 45+15
# 0.0843633333333 45
# 0.083702 30+15
# 0.0847226666667 30
# 0.083427 100+15
# 0.0839198 100
# 0.0834056 75+15
# 0.0839876 75

66

398:0.0835366


In [133]:
#train/test
params = {
    'objective': 'binary:logistic',
    'eta': 0.04,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': -1
}

predictors = list(set(basics_cols + predictors))
predictors = list(set(predictors) - set(rubbish_features))
len(predictors)

train = pd.concat([train, val])
xgbtrain = xgb.DMatrix(train[predictors], train[target])
model = xgb.train(params, xgbtrain, num_boost_round=600)

xgbtest = xgb.DMatrix(test[predictors])
y_pred = model.predict(xgbtest)

idx = pd.read_csv('resultWithHistory.txt', sep=' ')
idx.head()
idx['predicted_score'] = y_pred
idx.head()

idx[['instance_id', 'predicted_score']].to_csv(
    'resultWithHistory.txt',
    sep=' ',
    header=['instance_id', 'predicted_score'],
    index=False)